Em colaboração c as un de saúde do norte, a ESTG decidiu participar num projeto pra ajudar a identificar pacientes passíveis de terem um ataque cardíaco. Após um processo de recolha de dados (heart.csv) é agr necessário treinar um modelo de classificação, tarefa q ficou atribuída aos alunos de IA. 

| Variable Name | Description                                                                |
|---------------|----------------------------------------------------------------------------|
| age           | Age of the patient                                                         |
| sex           | Sex of the patient                                                         |
| exang         | Exercise induced angina (1 = yes; 0 = no)                                   |
| ca            | Number of major vessels (0-3)                                              |
| cp            | Chest Pain type chest pain type                                            |
|               | 1: typical angina                                                          |
|               | 2: atypical angina                                                         |
|               | 3: non-anginal pain                                                        |
|               | 4: asymptomatic                                                            |
| trtbps        | Resting blood pressure (in mm Hg)                                          |
| chol          | Cholesterol in mg/dl fetched via BMI sensor                                |
| fbs           | Fasting blood sugar > 120 mg/dl (1 = true; 0 = false)                       |
| rest_ecg      | Resting electrocardiographic results                                       |
|               | 0: normal                                                                  |
|               | 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or  |
|               | depression of > 0.05 mV)                                                   |
|               | 2: showing probable or definite left ventricular hypertrophy by Estes'     |
|               | criteria                                                                   |
| thalach       | Maximum heart rate achieved                                                |
| output        | 0= less chance of heart attack 1= more chance of heart attack              |

In [24]:
# pip install h2o
import h2o
from h2o.estimators import H2ORandomForestEstimator, H2OGeneralizedLinearEstimator

In [25]:
# Inicializa o sv H2O.
# Isso config um ambiente H2O local pra executar modelos de ML.
h2o.init()

# Carregar o dataset
data = h2o.import_file('heart.csv', header=1)

# Converter a coluna 'sex', 'exng', 'ca', 'cp', 'fbs', 'rest_ecg' e 'output' pra fatores, pois são categóricas
data['sex'] = data['sex'].asfactor()
data['exng'] = data['exng'].asfactor()
data['caa'] = data['caa'].asfactor()
data['cp'] = data['cp'].asfactor()
data['fbs'] = data['fbs'].asfactor()
data['restecg'] = data['restecg'].asfactor()
data['output'] = data['output'].asfactor()

Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,1 hour 8 mins
H2O_cluster_timezone:,Europe/Lisbon
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,4 months and 23 days
H2O_cluster_name:,H2O_from_python_Alice_Dias_vca5mq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.937 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Utilizando H2O treine um modelo de classificação pra o problema descrito. Uma vez q o dataset é relativamente pequeno, utilize uma abordagem c cross-validation.

In [26]:
# Dividir os dados em conj de treino e validação (cross-validation será usada)
train, valid = data.split_frame(ratios=[0.8], seed=42)

# Config e treinar o modelo de classificação c validação cruzada
heart_drf = H2ORandomForestEstimator(
    ntrees=50,
    max_depth=20,
    nfolds=5,  # Uso de 5-folds cross-validation
    seed=42,
    keep_cross_validation_predictions=True
)

heart_drf.train(x=train.columns[:-1], y="output", training_frame=train, validation_frame=valid)


drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_model_python_1742922984785_3580


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    50                 50                          29702                  8            13           10.14         33            54            42.62

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.1458752242992423
RMSE: 0.3819361521239411
LogLoss: 0.7100099598225366
Mean Per-Class Error: 0.20786163522012577
AUC: 0.8602725366876309
AUCPR: 0.8502777330116367
Gini: 0.7205450733752619

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4285714285714286
       0    1    Error    Rate
-----  ---  ---  -------  ------------
0      69   37   0.3491   (37.0/106.0)
1      9    126  0.0667   (9.0/135.0)
Total  78   163  0.1909   (46.0/241.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.428571     0.845638  92
max f2                       0.277778     0.915395  106
max f0point5                 0.675        0.825771  59
max accuracy                 0.428571     0.809129  92
max precision                0.954545     0.923077  10
max recall                   0.0666667    1         133
max specificity              1            0.981132  0
max absolute_mcc             0.428571     0.619865  92
max min_per_class_accuracy   0.595238     0.777778  69
max mean_per_class_accuracy  0.428571     0.792138  92
max tns                      1            104       0
max fns                      1            122       0
max fps                      0            106       143
max tps                      0.0666667    135       133
max tnr                      1            0.981132  0
max fnr                      1            0.903704  0
max fpr                      0            1         143
max tpr                      0.0666667    1         133

Gains/Lift Table: Avg response rate: 56.02 %, avg score: 56.25 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0622407                   1                  1.54716    1.54716            0.866667         1          0.866667                    1                   0.0962963       0.0962963                  54.716    54.716             0.0774284
2        0.107884                    0.954545           1.78519    1.64786            1                0.973947   0.923077                    0.988977            0.0814815       0.177778                   78.5185   64.7863            0.15891
3        0.153527                    0.923077           1.46061    1.59219            0.818182         0.939243   0.891892                    0.974191            0.0666667       0.244444                   46.0606   59.2192            0.206709
4        0.20332                     0.875              1.63642    1.60302            0.916667         0.892528   0.897959                    0.954192            0.0814815       0.325926                   63.642    60.3023            0.278756
5        0.307054                    0.8                1.57096    1.59219            0.88             0.832571   0.891892                    0.913104            0.16296

Analise os modelos intermédios, e verifique se há uma variância significativa na sua performance. 

In [27]:
# Avaliar o desempenho do modelo
performance = heart_drf.model_performance(valid)
print(performance)

# Analisar as métricas de cross-validation
cv_summary = heart_drf.cross_validation_metrics_summary().as_data_frame()
print(cv_summary)

ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.1124994462704384
RMSE: 0.3354093711726588
LogLoss: 0.35316487211934094
Mean Per-Class Error: 0.12916666666666665
AUC: 0.934375
AUCPR: 0.9421743993575991
Gini: 0.8687499999999999

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5800000000000001
       0    1    Error    Rate
-----  ---  ---  -------  ----------
0      28   4    0.125    (4.0/32.0)
1      4    26   0.1333   (4.0/30.0)
Total  32   30   0.129    (8.0/62.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.58         0.866667  22
max f2                       0.16         0.898204  37
max f0point5                 0.706667     0.909091  14
max accuracy                 0.63         0.870968  20
max precision                1            1         0
max recall                   0.16         1         37

Analise ainda as métricas de performance relevantes pra o caso, e indique, de forma fundamentada, se o modelo tem qualidade suficiente pra ser utilizado em produção.  

In [28]:
# Métricas adicionais
print("Acurácia: ", performance.accuracy())
print("AUC: ", performance.auc())
print("Recall: ", performance.recall())
print("Precision: ", performance.precision())

Acurácia:  [[0.63, 0.8709677419354839]]
AUC:  0.934375
Recall:  [[0.16000000000000003, 1.0]]
Precision:  [[1.0, 1.0]]
